In [ ]:
import os
import numpy as np
import pandas as pd
import random
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import metrics
import tensorflow as tf
from keras.preprocessing import image
from skimage import io
#attention
from keras.layers import GlobalAveragePooling2D, multiply, LocallyConnected2D, Lambda
from keras.models import Model


seed = 232
np.random.seed(seed)
tf.random.set_seed(seed)


In [ ]:
import sys
print(sys.version)

In [ ]:
#import tensorflow as tf
print(tf.__version__)

In [ ]:
import keras
print(keras.__version__)

In [ ]:
import zipfile

In [ ]:
#import os
import shutil
#import numpy as np

In [ ]:
import math

Data loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
Dataset = "data_byclass_jan20"

In [ ]:
with zipfile.ZipFile("/content/drive/My Drive/"+Dataset+".zip","r") as z:
    z.extractall(".")

In [ ]:
print(len(os.listdir("./boneage-training-dataset")))

In [ ]:
counts = 0
for folders, subfolders, filenames in os.walk("/content/drive/My Drive/CT/test/covid/"):
    for filename in filenames:
        counts=counts+1
print(counts)


In [ ]:
source_folder = './Gandhi_test6'
destination_folder = '/content/drive/My Drive/CT/test/covid/'


In [ ]:
count = 0
for folders, subfolders, filenames in os.walk(source_folder):
    for filename in filenames:
      #if filename in file_list:
      shutil.move(os.path.join(folders, filename), destination_folder)


In [ ]:
input_path = '/content/drive/My Drive/data_byclass_nov12/'

fig, ax = plt.subplots(3, 3, figsize=(15, 7))
ax = ax.ravel()
plt.tight_layout()

for i, _set in enumerate(['train', 'val', 'test']):
    set_path = input_path+_set
    ax[i].imshow(plt.imread(set_path+'/normal/'+os.listdir(set_path+'/normal')[0]), cmap='gray')
    ax[i].set_title('Set: {}, Condition: Normal'.format(_set))
    ax[i+3].imshow(plt.imread(set_path+'/pneumonia/'+os.listdir(set_path+'/pneumonia')[0]), cmap='gray')
    ax[i+3].set_title('Set: {}, Condition: Pneumonia'.format(_set))
    ax[i+6].imshow(plt.imread(set_path+'/covid/'+os.listdir(set_path+'/covid')[0]), cmap='gray')
    ax[i+6].set_title('Set: {}, Condition: Covid'.format(_set))

In [ ]:
input_path = '/content/drive/My Drive/CT/'

for _set in ['test']:
    n_normal = len(os.listdir(input_path + _set + '/normal'))
    n_infect = len(os.listdir(input_path + _set + '/pneumonia'))
    n_covid = len(os.listdir(input_path + _set + '/covid'))
    print('Set: {}, normal images: {}, pneumonia images: {}, covid images: {}'.format(_set, n_normal, n_infect, n_covid))

In [ ]:
input_path = '/content/drive/My Drive/CT/'


In [ ]:
import imghdr

Pre-processing Data

In [ ]:
def process_data(img_dims, batch_size):
    # Data generation objects

    #train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, vertical_flip=True)
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)
    """
    train_datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    zoom_range=(0.85,1.15),
    shear_range = 0.3,
    horizontal_flip=True,
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1)
    #val_datagen = ImageDataGenerator(rescale=1./255)
    """

    # This is fed to the network in the specified batch sizes and image dimensions
    train_gen = train_datagen.flow_from_directory(
    directory=input_path+'test',
    target_size=(img_dims, img_dims),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

    val_gen = val_datagen.flow_from_directory(
    directory=input_path+'test',
    target_size=(img_dims, img_dims),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

    # I will be making predictions off of the test set in one batch size
    # This is useful to be able to get the confusion matrix
    test_data = []
    test_labels = []
    """
    for cond in ['/normal/', '/pneumonia/', '/covid/']:
        for img in (os.listdir(input_path + 'test' + cond)):
            img = cv2.imread(input_path+'test'+cond+img)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (img_dims, img_dims))
            img = np.dstack([img, img, img])
            img = img.astype('float32') / 255
            if cond=='/normal/':
                label = 1
            elif cond=='/pneumonia/':
                label = 2
            elif cond== '/covid/':
                label = 0
            test_data.append(img)
            test_labels.append(label)

    test_data = np.array(test_data)
    #test_labels = np.array(test_labels)
    """

    #return val_gen
    return train_gen, val_gen, test_data, test_labels

In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
labels =['covid', 'normal', 'pneumonia']

In [ ]:
#duplicate
def process_data(img_dims, batch_size):
    # Data generation objects

    #train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, vertical_flip=True)
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)
    """
    train_datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    zoom_range=(0.85,1.15),
    shear_range = 0.3,
    horizontal_flip=True,
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1)
    #val_datagen = ImageDataGenerator(rescale=1./255)
    """

    # This is fed to the network in the specified batch sizes and image dimensions
    train_gen = train_datagen.flow_from_directory(
    directory=input_path+'test1',
    target_size=(img_dims, img_dims),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

    val_gen = val_datagen.flow_from_directory(
    directory=input_path+'test1',
    target_size=(img_dims, img_dims),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)
    """
    # I will be making predictions off of the test set in one batch size
    # This is useful to be able to get the confusion matrix
    test_data = []
    test_labels = []

    for cond in ['/normal/', '/pneumonia/', '/covid/']:
        for img in (os.listdir(input_path + 'test' + cond)):
            img = cv2.imread(input_path+'test'+cond+img)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (img_dims, img_dims))
            img = np.dstack([img, img, img])
            img = img.astype('float32') / 255
            if cond=='/normal/':
                label = 1
            elif cond=='/pneumonia/':
                label = 2
            elif cond== '/covid/':
                label = 0
            test_data.append(img)
            test_labels.append(label)

    test_data = np.array(test_data)
    test_labels = np.array(test_labels)
    """

    #return val_gen
    return train_gen, val_gen

In [ ]:
img_dims = 224
epochs = 1
batch_size = 8

#train_gen, val_gen, test_data, test_labels = process_data(img_dims, batch_size)
#train_gen, val_gen = process_data(img_dims, batch_size)

Building Models

In [ ]:
from tensorflow.keras.metrics import AUC, CategoricalCrossentropy


In [ ]:
inputs = Input(shape=(img_dims, img_dims, 3))

# First conv block
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
#x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Second conv block
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
#x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Third conv block
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
#x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Fourth conv block
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
#x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

# Fifth conv block
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
#x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

# FC layer
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
#x = Dropout(rate=0.7)(x)
x = Dense(units=128, activation='relu')(x)
#x = Dropout(rate=0.5)(x)
x = Dense(units=64, activation='relu')(x)
#x = Dropout(rate=0.3)(x)

# Output layer
output = Dense(units=3, activation='softmax')(x)

# Creating model and compiling
model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint(filepath='/content/drive/My Drive/best_weights_cnn1.hdf5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')

In [ ]:
#from ada
inputs = Input(shape=(img_dims, img_dims, 3))

# First conv block
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Second conv block
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Third conv block
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
# Fourth conv block
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

# Fifth conv block
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

#Sixth
x = SeparableConv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)
"""
#for experimenting seventh layer
#Seventh
x = SeparableConv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)
"""
# FC layer
x = Flatten()(x)
#x = Dense(units=1024, activation='relu')(x)
#x = Dropout(rate=0.7)(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(rate=0.7)(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(rate=0.5)(x)
x = Dense(units=64, activation='relu')(x)
x = Dropout(rate=0.3)(x)

# Output layer
output = Dense(units=3, activation='softmax')(x)
optimizer = Adam(learning_rate=5e-5,beta_1=0.9,beta_2=0.999,epsilon=1e-07)
# Creating model and compiling
model = Model(inputs=inputs, outputs=output)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint(filepath='./best_weights_ctdump.hdf5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')



In [ ]:
model.summary()

Attention mechanism plugged

In [ ]:
hidden = 16
inputs = Input(shape=(img_dims, img_dims, 3))

# First conv block

x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2,2))(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2,2))(x)
x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2,2))(x)
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2,2))(x)
x = BatchNormalization()(x)

# attention mechanism

attn_layer = Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'relu')(x)
attn_layer = Conv2D(8, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
attn_layer = LocallyConnected2D(1,
                                kernel_size = (1,1),
                                padding = 'valid',
                                activation = 'sigmoid')(attn_layer)
# fan it out to all of the channels
up_c2_w = np.ones((1, 1, 1, hidden))
up_c2 = Conv2D(hidden, kernel_size = (1,1), padding = 'same',
               activation = 'linear', use_bias = False, weights = [up_c2_w])
up_c2.trainable = False
attn_layer = up_c2(attn_layer)

mask_features = multiply([attn_layer, x])
gap_features = GlobalAveragePooling2D()(mask_features)
gap_mask = GlobalAveragePooling2D()(attn_layer)
# to account for missing values from the attention model
gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])

gap_dr = Dropout(0.5)(gap)
dr_steps = Dropout(0.25)(Dense(512, activation = 'relu')(gap_dr))
out_layer = Dense(3, activation = 'softmax')(dr_steps) # linear is what 16bit did
covid_model = Model(inputs = [inputs], outputs = [out_layer])

optimizer =Adam(lr=5e-5,beta_1=0.9,beta_2=0.999,epsilon=1e-07)



covid_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint(filepath='./best_weights_ctdump.hdf5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')



In [ ]:
covid_model.summary()

In [ ]:




x = SeparableConv2D(filters=2048, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=2048, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)


In [ ]:
model.summary()

Data Balancing

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
weights = compute_class_weight('balanced', np.unique(train_gen.classes), train_gen.classes)
cw = dict(zip( np.unique(train_gen.classes), weights))
print(cw)

In [ ]:
filename='log.csv'
history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)

In [ ]:
#, steps_per_epoch=val_gen.samples // batch_size,

In [ ]:
def fixed_generator(generator):
    for batch in generator:
        yield (batch, batch)

In [ ]:
hist = covid_model.fit(generator=fixed_generator(val_gen),
           epochs=1, validation_data=fixed_generator(val_gen), callbacks=[checkpoint,lr_reduce,history_logger])

In [ ]:
val_gen.class_indices

In [ ]:
img, lab = next(val_gen)

In [ ]:
val_gen.samples

In [ ]:
callbacks_list=[checkpoint,lr_reduce,history_logger]

In [ ]:
history = covid_model.fit(val_gen,epochs=5, validation_data=val_gen, shuffle=True,  verbose=2)

In [ ]:
# get the attention layer since it is the only one with a single output dim
for attn_layer in covid_model.layers:
    c_shape = attn_layer.get_output_shape_at(0)
    if len(c_shape)==4:
        if c_shape[-1]==1:
            print(attn_layer)
            break

In [ ]:
test_X, test_Y = next(val_gen)

In [ ]:
validation_steps=val_gen.samples // batch_size,

In [ ]:
param_range = np.arange(1,2,0.5)

In [ ]:
param_range.shape

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()

for i, met in enumerate(['accuracy', 'loss']):
    ax[i].plot(param_range,hist.history[met],color ='green')
    ax[i].plot(param_range,hist.history['val_'+ met],color ='red')
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

In [ ]:
plt.subplots(2, figsize=(7,7))
for i, met in enumerate(['accuracy', 'loss']):
  plt.plot(param_range, ax[i], label="Training score", color="red")
  plt.plot(param_range, test_mean, label="Cross-validation score", color="green")

#plt.fill_between(param_range, train_mean - train_std, train_mean + train_std, color="gray")
#plt.fill_between(param_range, test_mean - test_std, test_mean + test_std, color="gainsboro")

plt.title("Validation Curve With Random Forest")
plt.xlabel("Number Of Trees")
plt.ylabel("Accuracy Score")
plt.tight_layout()
plt.legend(loc="best")
plt.show()

In [ ]:
plt.subplots(1, figsize=(7,7))
for i, met in enumerate(['accuracy', 'loss']):
  plt.plot(param_range, ax[i], label="Training score", color="black")
  plt.plot(param_range, test_mean, label="Cross-validation score", color="dimgrey")

  plt.fill_between(param_range, train_mean - train_std, train_mean + train_std, color="gray")
  plt.fill_between(param_range, test_mean - test_std, test_mean + test_std, color="gainsboro")

  plt.title("Validation Curve With Random Forest")
  plt.xlabel("Number Of Trees")
  plt.ylabel("Accuracy Score")
  plt.tight_layout()
  plt.legend(loc="best")
  plt.show()


In [ ]:
model_loaded = Model(inputs=inputs, outputs=output)
model_loaded.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
#the best so far. don't change
model.load_weights('/content/drive/My Drive/best_weights_ctdump.hdf5')


Testing performance

In [ ]:
eval_datagen = ImageDataGenerator(rescale = 1.0/255)

test_generator = eval_datagen.flow_from_directory(
    '/content/drive/My Drive/data_byclass_jan20/test/',
    target_size = (img_dims, img_dims),
    batch_size = batch_size,
    class_mode = 'categorical', shuffle = False
)
"""
eval_result = model.evaluate(test_generator)
print('loss rate at evaluation data :', eval_result[0])
print('accuracy rate at evaluation data :', eval_result[1])
"""

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:


preds = model_loaded.predict(test_generator)
pred = np.argmax(preds, axis = 1)
acc = accuracy_score(test_generator.classes, pred)*100
cm = confusion_matrix(test_generator.classes, pred)
#tn, fp, fn, tp = cm.ravel()

print('CONFUSION MATRIX ------------------')
print(cm)

In [ ]:
preds = covid_model.predict(val_gen)
pred = np.argmax(preds, axis = 1)
acc = accuracy_score(val_gen.classes, pred)*100
cm = confusion_matrix(val_gen.classes, pred)
#tn, fp, fn, tp = cm.ravel()

print('CONFUSION MATRIX ------------------')
print(cm)
print(acc)

In [ ]:
#the best so far. don't change. with best_weights_ct2.hdf5
preds = model_loaded.predict(test_generator)
pred = np.argmax(preds, axis = 1)
acc = accuracy_score(test_generator.classes, pred)*100
cm = confusion_matrix(test_generator.classes, pred)
#tn, fp, fn, tp = cm.ravel()

print('CONFUSION MATRIX ------------------')
print(cm)

In [ ]:
#30
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

preds = model_loaded.predict(test_generator)
pred = np.argmax(preds, axis = 1)
acc = accuracy_score(test_generator.classes, pred)*100
cm = confusion_matrix(test_generator.classes, pred)
#tn, fp, fn, tp = cm.ravel()

print('CONFUSION MATRIX ------------------')
print(cm)

In [ ]:
#ctcnn1
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

preds = model_loaded.predict(test_generator)
pred = np.argmax(preds, axis = 1)
acc = accuracy_score(test_generator.classes, pred)*100
cm = confusion_matrix(test_generator.classes, pred)
#tn, fp, fn, tp = cm.ravel()

print('CONFUSION MATRIX ------------------')
print(cm)

In [ ]:
TN, FP, FN, TP, TN1, FP1, FN1, TP1, TN2 = cm.ravel()

In [ ]:
TP = 4692
FN = 1482

In [ ]:
FP = 352
TN = 19640

In [ ]:
print(TP/(TP+FN))

In [ ]:
print('Classification Report')
target_names = ['Covid', 'Normal', 'Pneumonia']
print(classification_report(test_generator.classes, pred, target_names=target_names))


In [ ]:
print('Classification Report')
target_names = ['Covid', 'Normal', 'Pneumonia']
print(classification_report(test_generator.classes, pred, target_names=target_names))


In [ ]:
#the best so far. don't change
print('Classification Report')
target_names = ['Covid', 'Normal', 'Pneumonia']
print(classification_report(test_generator.classes, pred, target_names=target_names))


In [ ]:
#30
print('Classification Report')
target_names = ['Covid', 'Normal', 'Pneumonia']
print(classification_report(test_generator.classes, pred, target_names=target_names))


In [ ]:
#ctcnn1
print('Classification Report')
target_names = ['Covid', 'Normal', 'Pneumonia']
print(classification_report(test_generator.classes, pred, target_names=target_names))

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1/255)

test_generator_gandhialone = test_datagen.flow_from_directory(
    '/content/drive/My Drive/CT/test/',
    target_size = (img_dims, img_dims),
    batch_size = batch_size,
    class_mode = 'categorical', shuffle = False
)


In [ ]:
preds = model.predict(test_generator_gandhialone)
pred = np.argmax(preds, axis = 1)
acc = accuracy_score(test_generator_gandhialone.classes, pred)*100
cm = confusion_matrix(test_generator_gandhialone.classes, pred)


In [ ]:
print('CONFUSION MATRIX ------------------')
print(cm)

In [ ]:
print(acc)

In [ ]:
print('Classification Report')
target_names = ['Covid', 'Normal', 'Pneumonia']
print(classification_report(test_generator_gandhialone.classes, pred, target_names=target_names))


In [ ]:
ppvs = [cm[i,i]/np.sum(cm[:,i]) if np.sum(cm[:,i]) else 0 for i in range(len(cm))]
print('PPV Covid: {0:.3f}, Normal: {1:.3f}, Pneumonia: {2:.3f}'.format(ppvs[0],
                                                                             ppvs[1],
                                                                             ppvs[2]))

In [ ]:
class_acc = [cm[i,i]/np.sum(cm[i,:]) if np.sum(cm[i,:]) else 0 for i in range(len(cm))]
print('Sens Covid: {0:.3f}, Normal: {1:.3f}, Pneumonia: {2:.3f}'.format(class_acc[0],
                                                                               class_acc[1],
                                                                               class_acc[2]))

In [ ]:
val_gen.class_indices

Visualization starts here

In [ ]:
pip install visual-logging

In [ ]:
import logging

import cv2
import numpy as np
import vlogging

logger = logging.getLogger("demo")
fh = logging.FileHandler('test.html', mode="w")

logger.setLevel(logging.DEBUG)
logger.addHandler(fh)

image = cv2.imread('/content/drive/My Drive/CT/test/covid/C19_603970_26.png')
logger.info(vlogging.VisualRecord('Sample Image', image, {'footnote': 'sample image'}, fmt='png'))

In [ ]:
from PIL import Image

In [ ]:
#import cv2
#import numpy as np
#import tensorflow as tf

IMAGE_PATH = '/content/drive/My Drive/CT/test/covid/C19_603970_26.png'
LAYER_NAME = 'separable_conv2d_9'
CAT_CLASS_INDEX = 0

img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(224, 224))
img = tf.keras.preprocessing.image.img_to_array(img)

#model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)

grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(LAYER_NAME).output, model.output])

with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(np.array([img]))
    loss = predictions[:, CAT_CLASS_INDEX]

output = conv_outputs[0]
grads = tape.gradient(loss, conv_outputs)[0]

gate_f = tf.cast(output > 0, 'float32')
gate_r = tf.cast(grads > 0, 'float32')
guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

weights = tf.reduce_mean(guided_grads, axis=(0, 1))

cam = np.ones(output.shape[0: 2], dtype = np.float32)

for i, w in enumerate(weights):
    cam += w * output[:, :, i]

cam = cv2.resize(cam.numpy(), (224, 224))
cam = np.maximum(cam, 0)
heatmap = (cam - cam.min()) / (cam.max() - cam.min())

cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

output_image = cv2.addWeighted(cv2.cvtColor(img.astype('uint8'), cv2.COLOR_BGR2RGB), 0.5, cam, 1, 0)
plt.imshow(output_image)
plt.show()
#cv2.imwrite('/content/drive/My Drive/cam.png', output_image)

In [ ]:
LAYER_NAME = 'separable_conv2d_19'


In [ ]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions



In [ ]:
model = VGG16(weights="imagenet")

In [ ]:
sunglasses= io.imread("https://ae01.alicdn.com/kf/HTB1wnD8bcrrK1RjSspaq6AREXXaR/MK100-Tony-Stark-Doll-Head-Carved-Glasses-Seamless-Flexible-Body-1-6-Action-Figure-Scale-Model.jpg_q50.jpg")



In [ ]:
IMAGE_PATH = '/content/drive/My Drive/CT/test/covid/C19_603970_26.png'
LAYER_NAME = 'separable_conv2d_9'
CAT_CLASS_INDEX = 0

img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(224, 224))
img = tf.keras.preprocessing.image.img_to_array(img)



In [ ]:
sunglasses = cv2.resize(sunglasses, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)

x = image.img_to_array(sunglasses)

x = np.expand_dims(x, axis=0)

x = preprocess_input(x)



In [ ]:
x = np.expand_dims(img, axis=0)


In [ ]:
#Prediction

preds = model.predict(x)

class_output = model.output[:, 837]

last_conv_layer = model.get_layer("block5_conv3")



In [ ]:
preds = model.predict(x)

class_output = model.output

last_conv_layer = model.get_layer(LAYER_NAME)


In [ ]:
print(class_output)

In [ ]:
tf.compat.v1.disable_eager_execution()
"""
with tf.GradientTape() as tape:
    #predictions = model.predict(x)
    predictions = model(x)
    loss = predictions[:, 837]
    conv_outputs = last_conv_layer.output
    #loss = predictions[:, CAT_CLASS_INDEX]

output = conv_outputs[0]
grads = tape.gradient(loss, last_conv_layer.output)
"""
grads = K.gradients(class_output, last_conv_layer.output)[0]

print(grads.shape)

pooled_grads = K.mean(grads, axis=(0, 1, 2))

print(pooled_grads.shape)

iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

pooled_grads_value, conv_layer_output_value = iterate([x])

for i in range(512):

  conv_layer_output_value[:, :, i] *= pooled_grads_value[i]



In [ ]:
#img1 = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(224, 224))
img1 = cv2.resize(image.load_img(IMAGE_PATH), dsize=(224, 224), interpolation=cv2.INTER_CUBIC)

In [ ]:
heatmap = np.mean(conv_layer_output_value, axis = -1)

print(conv_layer_output_value.shape)

print(heatmap.shape)

heatmap = np.maximum(heatmap, 0)

heatmap /= np.max(heatmap)

heatmap = cv2.resize(heatmap, (sunglasses.shape[1], sunglasses.shape[0]))

heatmap = np.uint8(255 * heatmap)

heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

superimposed_img = cv2.addWeighted(sunglasses, 0.5, heatmap, 0.5, 0)

from google.colab.patches import cv2_imshow

cv2_imshow(heatmap)



In [ ]:
heatmap = np.mean(conv_layer_output_value, axis = -1)

print(conv_layer_output_value.shape)

print(heatmap.shape)

heatmap = np.maximum(heatmap, 0)

heatmap /= np.max(heatmap)

heatmap = cv2.resize(heatmap, (sunglasses.shape[1], sunglasses.shape[0]))

heatmap = np.uint8(255 * heatmap)

heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

superimposed_img = cv2.addWeighted(sunglasses, 0.5, heatmap, 0.5, 0)

from google.colab.patches import cv2_imshow

cv2_imshow(heatmap)



In [ ]:

predict = model.predict(np.array([img]))
#predict = model.predict(test_generator)
target_class = np.argmax(predict[0])
print("Target Class = ", target_class, "corresponding to:", predict, "Obese is [0., 1.]")

#original code
#print("The real value is",y_ann_data_encoded[example_index])

#code for running standalone
print(test_generator.classes[0])

In [ ]:
last_conv = model.get_layer('separable_conv2d_9') #last_conv= model.layers[8]
grad_model = tf.keras.models.Model([model.inputs], [last_conv.output, model.output])

with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(np.array([img]))
    loss = predictions[:, target_class]

In [ ]:
output = conv_outputs[0] #activations maps from last conv layer
grads = tape.gradient(loss, conv_outputs)[0] #function to obtain gradients from last conv layer

print("grads shape:", grads.shape)
print("Model output (loss for the target class):", loss.shape)
print("Output froom lat conv layer", conv_outputs.shape)


In [ ]:
activation_map = output[:,0]
plt.plot(activation_map)
plt.show()

Activation_map_image = [activation_map,activation_map,activation_map,activation_map,activation_map,
                        activation_map,activation_map,activation_map,activation_map,activation_map,
                        activation_map,activation_map,activation_map,activation_map,activation_map]
#plt.imshow(Activation_map_image) # vmin=-1, vmax=1) #cmap='gray
#plt.colorbar()

#c=a.numpy()
#c=a.numpy().reshape(1, -1)

activation_map[0:15]


In [ ]:
gradient = grads[:,0]

plt.plot(gradient)
plt.show()

gradient_map_image = [gradient,gradient,gradient,gradient,gradient,
                        gradient,gradient,gradient,gradient,gradient,
                        gradient,gradient,gradient,gradient,gradient]
#plt.imshow(gradient_map_image) # vmin=-1, vmax=1) #cmap='gray
#plt.colorbar()


In [ ]:
# Apply guided backpropagation
guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

guide_grads_example = guided_grads[:,0]
plt.plot(guide_grads_example)
plt.show()

guided_grads_image = [guide_grads_example,guide_grads_example,guide_grads_example, guide_grads_example,guide_grads_example,
                        guide_grads_example,guide_grads_example,guide_grads_example, guide_grads_example,guide_grads_example,
                        guide_grads_example,guide_grads_example,guide_grads_example, guide_grads_example,guide_grads_example]
plt.imshow(guided_grads_image) # vmin=-1, vmax=1) #cmap='gray
plt.colorbar()


In [ ]:
import numpy as np
import tensorflow as tf

layers_name = ['conv2D_1']
IMAGE_PATH = '/content/drive/My Drive/CT/test/covid/C19_603970_26.png'

# Model to examine
#model = tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=True)

# Image to pass as input
img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(224, 224))
img = tf.keras.preprocessing.image.img_to_array(img)

# Get the outputs of layers we want to inspect
outputs = [
    layer.output for layer in model.layers
    if layer.name in layers_name
]

# Create a connection between the input and those target outputs
activations_model = tf.keras.models.Model(model.inputs, outputs=outputs)
activations_model.compile(optimizer='adam', loss='categorical_crossentropy')

# Get their outputs
activations_1 = activations_model.predict(np.array([img]))

In [ ]:
from __future__ import print_function, division

from math import sqrt

from scipy.special import ndtri

def _proportion_confidence_interval(r, n, z):
    """Compute confidence interval for a proportion.

    Follows notation described on pages 46--47 of [1].

    References
    ----------
    [1] R. G. Newcombe and D. G. Altman, Proportions and their differences, in Statisics
    with Confidence: Confidence intervals and statisctical guidelines, 2nd Ed., D. G. Altman,
    D. Machin, T. N. Bryant and M. J. Gardner (Eds.), pp. 45-57, BMJ Books, 2000.
    """

    A = 2*r + z**2
    B = z*sqrt(z**2 + 4*r*(1 - r/n))
    C = 2*(n + z**2)
    return ((A-B)/C, (A+B)/C)

def sensitivity_and_specificity_with_confidence_intervals(TP, FP, FN, TN, alpha=0.95):
    """Compute confidence intervals for sensitivity and specificity using Wilson's method.

    This method does not rely on a normal approximation and results in accurate
    confidence intervals even for small sample sizes.

    Parameters
    ----------
    TP : int
        Number of true positives
    FP : int
        Number of false positives
    FN : int
        Number of false negatives
    TN : int
        Number of true negatives
    alpha : float, optional
        Desired confidence. Defaults to 0.95, which yields a 95% confidence interval.

    Returns
    -------
    sensitivity_point_estimate : float
        Numerical estimate of the test sensitivity
    specificity_point_estimate : float
        Numerical estimate of the test specificity
    sensitivity_confidence_interval : Tuple (float, float)
        Lower and upper bounds on the alpha confidence interval for sensitivity
    specificity_confidence_interval
        Lower and upper bounds on the alpha confidence interval for specificity

    References
    ----------
    [1] R. G. Newcombe and D. G. Altman, Proportions and their differences, in Statisics
    with Confidence: Confidence intervals and statisctical guidelines, 2nd Ed., D. G. Altman,
    D. Machin, T. N. Bryant and M. J. Gardner (Eds.), pp. 45-57, BMJ Books, 2000.
    [2] E. B. Wilson, Probable inference, the law of succession, and statistical inference,
    J Am Stat Assoc 22:209-12, 1927.
    """

    #
    z = -ndtri((1.0-alpha)/2)

    # Compute sensitivity using method described in [1]
    sensitivity_point_estimate = TP/(TP + FN)
    sensitivity_confidence_interval = _proportion_confidence_interval(TP, TP + FN, z)

    # Compute specificity using method described in [1]
    #specificity_point_estimate = TN/(TN + FP)
    #specificity_confidence_interval = _proportion_confidence_interval(TN, TN + FP, z)

    # Compute PPV using method described in [1]
    ppv_point_estimate = TP/(TP + FP)
    ppv_confidence_interval = _proportion_confidence_interval(TP, TP + FP, z)

    return sensitivity_point_estimate, ppv_point_estimate, sensitivity_confidence_interval, ppv_confidence_interval

for a in [0.95, 0.99]:
    sensitivity_point_estimate, specificity_point_estimate, \
        sensitivity_confidence_interval, specificity_confidence_interval \
        = sensitivity_and_specificity_with_confidence_intervals(6502, 2924, 893, 21417, alpha=a)
    print("Sensitivity: %f, Specificity: %f" %(sensitivity_point_estimate, specificity_point_estimate))
    print("alpha = %f CI for sensitivity:"%a, sensitivity_confidence_interval)
    print("alpha = %f CI for specificity:"%a, specificity_confidence_interval)
    print("")


In [ ]:
#For Reference
"""
for a in [0., 0.5, 0.9, 0.95, 0.99, 0.999999]:
    sensitivity_point_estimate, specificity_point_estimate, \
        sensitivity_confidence_interval, specificity_confidence_interval \
        = sensitivity_and_specificity_with_confidence_intervals(6502, 1612, 893, 16807, alpha=a)
    print("Sensitivity: %f, Specificity: %f" %(sensitivity_point_estimate, specificity_point_estimate))
    print("alpha = %f CI for sensitivity:"%a, sensitivity_confidence_interval)
    print("alpha = %f CI for specificity:"%a, specificity_confidence_interval)
    print("")
"""

In [ ]:
import numpy as np # import libraries
from statsmodels.stats import contingency_tables
import pandas as pd
from scipy.stats import binom

In [ ]:
contingency_tables.mcnemar(cm,exact=True).pvalue

In [ ]:
def parse_image(file_name):
  """
  This function downloads and standardizes input JPEG images for the
    inception_v1 model. Its applies the following processing:
    - Reads JPG file.
    - Decodes JPG file into colored image.
    - Converts data type to standard tf.float32.
    - Resizes image to expected Inception V1 input dimension of
      (224, 224, 3) with preserved aspect ratio. E.g. don't stretch image.
    - Pad image to (224, 224, 3) shape with black pixels.
  Args:
    file_name(str): Direct URL path to the JPG image.
  Returns:
    image(Tensor): A Tensor of floats with shape (224, 224, 3).
    label(str): A text label for display above the image.
  """
  image = tf.io.read_file(file_name)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, (224, 224), preserve_aspect_ratio=True)
  image = tf.image.resize_with_pad(image, target_height=224, target_width=224)

  return image

In [ ]:
input_path = '/content/drive/My Drive/CT/'

img_path = os.path.join(input_path, 'train', 'covid','volume-covid19-A-0684-0022.png')


In [ ]:
img = {'volume-covid19-A-0684-0022':img_path}

In [ ]:
img_name_tensors = {name: parse_image(img_path) for (name, img_path) in img.items()}

In [ ]:
plt.figure(figsize=(14,14))
for n, (name, img_tensors) in enumerate(img_name_tensors.items()):
  ax = plt.subplot(3,3,n+1)
  ax.imshow(img_tensors)
  ax.set_title(name)
  ax.axis('off')
plt.tight_layout()

In [ ]:
# stack images into a batch for processing.
image_titles = tf.convert_to_tensor(list(img_name_tensors.keys()))
image_batch = tf.convert_to_tensor(list(img_name_tensors.values()))
image_batch.shape

In [ ]:
def top_k_predictions_scores_labels(model, img, label_vocab, top_k=3):
  """
  Args:
    model(tf.keras.Model): Trained Keras model.
    img(tf.Tensor): A 4D tensor of floats with the shape
      (img_n, img_height, img_width, 3).
    label_vocab(numpy.ndarray): An array of strings with shape (1001,).
    top_k(int): Number of results to return.
  Returns:
    k_predictions_idx(tf.Tensor): A tf.Tensor [n_images, top_k] of tf.int32
      prediction indicies.
    k_predictions_proba(tf.Tensor): A tf.Tensor [n_images, top_k] of tf.float32
      prediction probabilities..
    k_predictions_label(tf.Tensor): A tf.Tensor [n_images, top_k] of tf.string
      prediction labels.
  """
  # These are logits (unnormalized scores).
  predictions = model(img)
  # Convert logits into probabilities.
  predictions_proba = tf.nn.softmax(predictions, axis=-1)
  #predictions_proba = tf.nn.sigmoid(predictions)
  #print(predictions_proba)
  # Filter top k prediction probabilities and indices.
  k_predictions_proba, k_predictions_idx = tf.math.top_k(
      input=predictions_proba, k=top_k)
  # Lookup top k prediction labels in label_vocab array.
  k_predictions_label = tf.convert_to_tensor(
      label_vocab[k_predictions_idx.numpy()],
      dtype=tf.string)

  return k_predictions_idx, k_predictions_label, k_predictions_proba

In [ ]:
def plot_img_predictions(model, img, img_titles, label_vocab, top_k=3):
  """Plot images with top_k predictions.
  Args:
    model(tf.keras.Model): Trained Keras model.
    img(Tensor): A 4D Tensor of floats with the shape
      (img_n, img_height, img_width, 3).
    img_titles(Tensor): A Tensor of strings with the shape
      (img_n, img_height, img_width, 3).
    label_vocab(numpy.ndarray): An array of strings with shape (1001,).
    top_k(int): Number of results to return.
  Returns:
    fig(matplotlib.pyplot.figure): fig object to utilize for displaying, saving
      plots.
  """
  pred_idx, pred_label, pred_proba = \
  top_k_predictions_scores_labels(
      model=model,
      img=img,
      label_vocab=label_vocab,
      top_k=top_k)

  img_arr = img.numpy()
  title_arr = img_titles.numpy()
  pred_idx_arr = pred_idx.numpy()
  pred_label_arr = pred_label.numpy()
  pred_proba_arr = pred_proba.numpy()

  n_rows = img_arr.shape[0]
  # Preserve image height by converting pixels to inches based on dpi.
  size = n_rows * (224 // 48)

  fig, axs = plt.subplots(nrows=img_arr.shape[0], ncols=1, figsize=(size, size), squeeze=False)
  for idx, image in enumerate(img_arr):
    axs[idx, 0].imshow(image)
    axs[idx, 0].set_title(title_arr[idx].decode('utf-8'), fontweight='bold')
    axs[idx, 0].axis('off')
    for k in range(top_k):
      k_idx = pred_idx_arr[idx][k]
      k_label = pred_label_arr[idx][k].decode('utf-8')
      k_proba = pred_proba_arr[idx][k]
      if k==0:
        s = 'Prediction {:}: ({:}-{:}) Score: {:.1%}'.format(k+1, k_idx, k_label, k_proba)
        axs[idx, 0].text(244 + size, 102+(k*40), s, fontsize=12, fontweight='bold')
      else:
        s = 'Prediction {:}: ({:}-{:}) Score: {:.1%}'.format(k+1, k_idx, k_label, k_proba)
        axs[idx, 0].text(244 + size, 102+(k*20), s, fontsize=12)

  plt.tight_layout()

  return fig


In [ ]:
test_labels = np.array(test_generator.classes)

In [ ]:
test_labels[1]

In [ ]:
_ = plot_img_predictions(
    model=model,
    img=image_batch,
    img_titles=image_titles,
    label_vocab=test_labels[0],
    top_k=3
    )

In [ ]:
# name_baseline_tensors. Set random seed for reproducibility of random baseline image and associated attributions.
tf.random.set_seed(42)
name_baseline_tensors = {
    'Baseline Image: Black': tf.zeros(shape=(224,224,3)),
    'Baseline Image: Random': tf.random.uniform(shape=(224,224,3), minval=0.0, maxval=1.0),
    'Baseline Image: White': tf.ones(shape=(224,224,3)),
}

In [ ]:
plt.figure(figsize=(12,12))
for n, (name, baseline_tensor) in enumerate(name_baseline_tensors.items()):
  ax = plt.subplot(1,3,n+1)
  ax.imshow(baseline_tensor)
  ax.set_title(name)
  ax.axis('off')
plt.tight_layout()

In [ ]:
m_steps=20
alphas = tf.linspace(start=0.0, stop=1.0, num=m_steps+1)

In [ ]:
def generate_path_inputs(baseline,
                         input,
                         alphas):
  """Generate m interpolated inputs between baseline and input features.
  Args:
    baseline(Tensor): A 3D image tensor of floats with the shape
      (img_height, img_width, 3).
    input(Tensor): A 3D image tensor of floats with the shape
      (img_height, img_width, 3).
    alphas(Tensor): A 1D tensor of uniformly spaced floats with the shape
      (m_steps,).
  Returns:
    path_inputs(Tensor): A 4D tensor of floats with the shape
      (m_steps, img_height, img_width, 3).
  """
  # Expand dimensions for vectorized computation of interpolations.
  alphas_x = alphas[:, tf.newaxis, tf.newaxis, tf.newaxis]
  baseline_x = tf.expand_dims(baseline, axis=0)
  input_x = tf.expand_dims(input, axis=0)
  delta = input_x - baseline_x
  path_inputs = baseline_x +  alphas_x * delta

  return path_inputs

In [ ]:
path_inputs = generate_path_inputs(
    baseline=name_baseline_tensors['Baseline Image: Black'],
    input=img_name_tensors['C19_603970_26'],
    alphas=alphas)

In [ ]:
path_inputs.shape

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=5, squeeze=False, figsize=(24, 24))

axs[0,0].set_title('Baseline \n alpha: {:.2f}'.format(alphas[0]))
axs[0,0].imshow(path_inputs[0])
axs[0,0].axis('off')

axs[0,1].set_title('=> Interpolated Image # 1 \n alpha: {:.2f}'.format(alphas[1]))
axs[0,1].imshow(path_inputs[1])
axs[0,1].axis('off')

axs[0,2].set_title('=> Interpolated Image # 2 \n alpha: {:.2f}'.format(alphas[2]))
axs[0,2].imshow(path_inputs[2])
axs[0,2].axis('off')

axs[0,3].set_title('... => Interpolated Image # 10 \n alpha: {:.2f}'.format(alphas[10]))
axs[0,3].imshow(path_inputs[10])
axs[0,3].axis('off')

axs[0,4].set_title('... => Input Image \n alpha: {:.2f}'.format(alphas[-1]))
axs[0,4].imshow(path_inputs[-1])
axs[0,4].axis('off')

plt.tight_layout();

In [ ]:
def compute_gradients(model, path_inputs, target_class_idx):
  """Compute gradients of model predicted probabilties with respect to inputs.
  Args:
    mode(tf.keras.Model): Trained Keras model.
    path_inputs(Tensor): A 4D tensor of floats with the shape
      (m_steps, img_height, img_width, 3).
    target_class_idx(Tensor): A 0D tensor of an int corresponding to the correct
      ImageNet target class index.
  Returns:
    gradients(Tensor): A 4D tensor of floats with the shape
      (m_steps, img_height, img_width, 3).
  """
  with tf.GradientTape() as tape:
    tape.watch(path_inputs)
    predictions = model(path_inputs)
    # Note: IG requires softmax probabilities; converting Inception V1 logits.
    outputs = tf.nn.softmax(predictions, axis=-1)[:, target_class_idx]
    #outputs = tf.nn.sigmoid(predictions)[:, target_class_idx]
    print(outputs)

  gradients = tape.gradient(outputs, path_inputs)

  return gradients

In [ ]:
path_gradients = compute_gradients(
    model=model,
    path_inputs=path_inputs,
    target_class_idx=0)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=5, squeeze=False, figsize=(24, 24))
for i in range(5):
  axs[0,i].imshow(tf.cast(255 * path_gradients[i], tf.uint8), cmap=plt.cm.inferno)
  axs[0,i].axis('off')
plt.tight_layout()

In [ ]:
pred = model(path_inputs)
pred_proba = tf.nn.softmax(pred, axis=-1)[:, 0]
#pred_proba = tf.nn.sigmoid(pred)[:, 0]

plt.figure(figsize=(10,4))
ax1 = plt.subplot(1,2,1)
ax1.plot(alphas, pred_proba)
ax1.set_title('Target class predicted probability over alpha')
ax1.set_ylabel('model p(target class)')
ax1.set_xlabel('alpha')
ax1.set_ylim([0,1])

ax2 = plt.subplot(1,2,2)
# Average across interpolation steps
average_grads = tf.math.reduce_mean(path_gradients, axis=[1,2,3])
# Normalize average gradients to 0 to 1 scale. E.g. (x - min(x))/(max(x)-min(x))
average_grads_norm = (average_grads-tf.math.reduce_min(average_grads))/(tf.math.reduce_max(average_grads)-tf.reduce_min(average_grads))
ax2.plot(alphas, average_grads_norm)
ax2.set_title('Average pixel gradients (normalized) over alpha')
ax2.set_ylabel('Average pixel gradients')
ax2.set_xlabel('alpha')
ax2.set_ylim([0,1]);

In [ ]:
def plot_riemann_sums(fn, start_val, end_val, m_steps=10):
  """
  Plot Riemann Sum integral approximations for single variable functions.
  Args:
    fn(function): Any single variable function.
    start_val(int): Minimum function value constraint.
    end_val(int): Maximum function value constraint.
    m_steps(int): Linear interpolation steps for approximation.
  Returns:
    fig(matplotlib.pyplot.figure): fig object to utilize for displaying, saving
      plots.
  """
  # fn plot args
  x = tf.linspace(start_val, end_val, m_steps**2+1)
  y = fn(x)

  fig = plt.figure(figsize=(16,4))

  # Left Riemann Sum
  lr_ax = plt.subplot(1,4,1)
  lr_ax.plot(x, y)
  lr_x = tf.linspace(0.0, 1.0, m_steps+1)
  lr_point = lr_x[:-1]
  lr_height = fn(lr_x[:-1])
  lr_ax.plot(lr_point, lr_height, 'b.', markersize=10)
  lr_ax.bar(lr_point, lr_height, width=(end_val-start_val)/m_steps, alpha=0.2, align='edge', edgecolor='b')
  lr_ax.set_title('Left Riemann Sum \n m_steps = {}'.format(m_steps))
  lr_ax.set_xlabel('alpha')
  # Right Riemann Sum
  rr_ax = plt.subplot(1,4,2)
  rr_ax.plot(x, y)
  rr_x = tf.linspace(0.0, 1.0, m_steps+1)
  rr_point = rr_x[1:]
  rr_height = fn(rr_x[1:])
  rr_ax.plot(rr_point, rr_height, 'b.', markersize=10)
  rr_ax.bar(rr_point, rr_height, width=-(end_val-start_val)/m_steps, alpha=0.2, align='edge', edgecolor='b')
  rr_ax.set_title('Right Riemann Sum \n m_steps = {}'.format(m_steps))
  rr_ax.set_xlabel('alpha')
  # Midpoint Riemann Sum
  mr_ax = plt.subplot(1,4,3)
  mr_ax.plot(x, y)
  mr_x = tf.linspace(0.0, 1.0, m_steps+1)
  mr_point = (mr_x[:-1] + mr_x[1:])/2
  mr_height = fn(mr_point)
  mr_ax.plot(mr_point, mr_height, 'b.', markersize=10)
  mr_ax.bar(mr_point, mr_height, width=(end_val-start_val)/m_steps, alpha=0.2, edgecolor='b')
  mr_ax.set_title('Midpoint Riemann Sum \n m_steps = {}'.format(m_steps))
  mr_ax.set_xlabel('alpha')
  # Trapezoidal Riemann Sum
  tp_ax = plt.subplot(1,4,4)
  tp_ax.plot(x, y)
  tp_x = tf.linspace(0.0, 1.0, m_steps+1)
  tp_y = fn(tp_x)
  for i in range(m_steps):
    xs = [tp_x[i], tp_x[i], tp_x[i+1], tp_x[i+1]]
    ys = [0, tp_y[i], tp_y[i+1], 0]
    tp_ax.plot(tp_x,tp_y,'b.',markersize=10)
    tp_ax.fill_between(xs, ys, color='C0', edgecolor='blue', alpha=0.2)
  tp_ax.set_title('Trapezoidal Riemann Sum \n m_steps = {}'.format(m_steps))
  tp_ax.set_xlabel('alpha')

  return fig

In [ ]:
_ = plot_riemann_sums(lambda x: tf.math.sin(x*math.pi), 0.0, 1.0, m_steps=5)

In [ ]:
_ = plot_riemann_sums(lambda x: tf.math.sin(x*math.pi), 0.0, 1.0, m_steps=10)

In [ ]:
def generate_alphas(m_steps=50,
                    method='riemann_trapezoidal'):
  """
  Args:
    m_steps(Tensor): A 0D tensor of an int corresponding to the number of linear
      interpolation steps for computing an approximate integral. Default is 50.
    method(str): A string representing the integral approximation method. The
      following methods are implemented:
      - riemann_trapezoidal(default)
      - riemann_left
      - riemann_midpoint
      - riemann_right
  Returns:
    alphas(Tensor): A 1D tensor of uniformly spaced floats with the shape
      (m_steps,).
  """
  m_steps_float = tf.cast(m_steps, float) # cast to float for division operations.

  if method == 'riemann_trapezoidal':
    alphas = tf.linspace(0.0, 1.0, m_steps+1) # needed to make m_steps intervals.
  elif method == 'riemann_left':
    alphas = tf.linspace(0.0, 1.0 - (1.0 / m_steps_float), m_steps)
  elif method == 'riemann_midpoint':
    alphas = tf.linspace(1.0 / (2.0 * m_steps_float), 1.0 - 1.0 / (2.0 * m_steps_float), m_steps)
  elif method == 'riemann_right':
    alphas = tf.linspace(1.0 / m_steps_float, 1.0, m_steps)
  else:
    raise AssertionError("Provided Riemann approximation method is not valid.")

  return alphas

In [ ]:
alphas = generate_alphas(m_steps=20, method='riemann_trapezoidal')

In [ ]:
def integral_approximation(gradients,
                           method='riemann_trapezoidal'):
  """Compute numerical approximation of integral from gradients.

  Args:
    gradients(Tensor): A 4D tensor of floats with the shape
      (m_steps, img_height, img_width, 3).
    method(str): A string representing the integral approximation method. The
      following methods are implemented:
      - riemann_trapezoidal(default)
      - riemann_left
      - riemann_midpoint
      - riemann_right
  Returns:
    integrated_gradients(Tensor): A 3D tensor of floats with the shape
      (img_height, img_width, 3).
  """
  if method == 'riemann_trapezoidal':
    grads = (gradients[:-1] + gradients[1:]) / tf.constant(2.0)
  elif method == 'riemann_left':
    grads = gradients
  elif method == 'riemann_midpoint':
    grads = gradients
  elif method == 'riemann_right':
    grads = gradients
  else:
    raise AssertionError("Provided Riemann approximation method is not valid.")

  # Average integration approximation.
  integrated_gradients = tf.math.reduce_mean(grads, axis=0)


  return integrated_gradients

In [ ]:
ig = integral_approximation(
    gradients=path_gradients,
    method='riemann_trapezoidal')

In [ ]:
@tf.function
def integrated_gradients(model,
                         baseline,
                         input,
                         target_class_idx,
                         m_steps=50,
                         method='riemann_trapezoidal',
                         batch_size=32
                        ):
  """
  Args:
    model(keras.Model): A trained model to generate predictions and inspect.
    baseline(Tensor): A 3D image tensor with the shape
      (image_height, image_width, 3) with the same shape as the input tensor.
    input(Tensor): A 3D image tensor with the shape
      (image_height, image_width, 3).
    target_class_idx(Tensor): An integer that corresponds to the correct
      ImageNet class index in the model's output predictions tensor. Default
        value is 50 steps.
    m_steps(Tensor): A 0D tensor of an integer corresponding to the number of
      linear interpolation steps for computing an approximate integral.
    method(str): A string representing the integral approximation method. The
      following methods are implemented:
      - riemann_trapezoidal(default)
      - riemann_left
      - riemann_midpoint
      - riemann_right
    batch_size(Tensor): A 0D tensor of an integer corresponding to a batch
      size for alpha to scale computation and prevent OOM errors. Note: needs to
      be tf.int64 and shoud be < m_steps. Default value is 32.
  Returns:
    integrated_gradients(Tensor): A 3D tensor of floats with the same
      shape as the input tensor (image_height, image_width, 3).
  """

  # 1. Generate alphas.
  alphas = generate_alphas(m_steps=m_steps,
                           method=method)

  # Initialize TensorArray outside loop to collect gradients. Note: this data structure
  # is similar to a Python list but more performant and supports backpropogation.
  # See https://www.tensorflow.org/api_docs/python/tf/TensorArray for additional details.
  gradient_batches = tf.TensorArray(tf.float32, size=m_steps+1)

  # Iterate alphas range and batch computation for speed, memory efficiency, and scaling to larger m_steps.
  # Note: this implementation opted for lightweight tf.range iteration with @tf.function.
  # Alternatively, you could also use tf.data, which adds performance overhead for the IG
  # algorithm but provides more functionality for working with tensors and image data pipelines.
  for alpha in tf.range(0, len(alphas), batch_size):
    from_ = alpha
    to = tf.minimum(from_ + batch_size, len(alphas))
    alpha_batch = alphas[from_:to]

    # 2. Generate interpolated inputs between baseline and input.
    interpolated_path_input_batch = generate_path_inputs(baseline=baseline,
                                                         input=input,
                                                         alphas=alpha_batch)

    # 3. Compute gradients between model outputs and interpolated inputs.
    gradient_batch = compute_gradients(model=model,
                                       path_inputs=interpolated_path_input_batch,
                                       target_class_idx=target_class_idx)

    # Write batch indices and gradients to TensorArray. Note: writing batch indices with
    # scatter() allows for uneven batch sizes. Note: this operation is similar to a Python list extend().
    # See https://www.tensorflow.org/api_docs/python/tf/TensorArray#scatter for additional details.
    gradient_batches = gradient_batches.scatter(tf.range(from_, to), gradient_batch)

  # Stack path gradients together row-wise into single tensor.
  total_gradients = gradient_batches.stack()

  # 4. Integral approximation through averaging gradients.
  avg_gradients = integral_approximation(gradients=total_gradients,
                                         method=method)

  # 5. Scale integrated gradients with respect to input.
  integrated_gradients = (input - baseline) * avg_gradients

  return integrated_gradients

In [ ]:
ig_attributions = integrated_gradients(model=model,
                          baseline=name_baseline_tensors['Baseline Image: Black'],
                          input=img_name_tensors['C19_603970_26'],
                          target_class_idx=0,
                          m_steps=55,
                          method='riemann_trapezoidal')

In [ ]:
def convergence_check(model, attributions, baseline, input, target_class_idx):
  """
  Args:
    model(keras.Model): A trained model to generate predictions and inspect.
    baseline(Tensor): A 3D image tensor with the shape
      (image_height, image_width, 3) with the same shape as the input tensor.
    input(Tensor): A 3D image tensor with the shape
      (image_height, image_width, 3).
    target_class_idx(Tensor): An integer that corresponds to the correct
      ImageNet class index in the model's output predictions tensor. Default
        value is 50 steps.
  Returns:
    (none): Prints scores and convergence delta to sys.stdout.
  """
  # Your model's prediction on the baseline tensor. Ideally, the baseline score
  # should be close to zero.
  baseline_prediction = model(tf.expand_dims(baseline, 0))
  baseline_score = tf.nn.softmax(tf.squeeze(baseline_prediction))[target_class_idx]
  #baseline_score = tf.nn.sigmoid(baseline_prediction)
  # Your model's prediction and score on the input tensor.
  input_prediction = model(tf.expand_dims(input, 0))
  input_score = tf.nn.softmax(tf.squeeze(input_prediction))[target_class_idx]
  #input_score = tf.nn.sigmoid(input_prediction)
  # Sum of your IG prediction attributions.
  ig_score = tf.math.reduce_sum(attributions)
  delta = ig_score - (input_score - baseline_score)
  try:
    # Test your IG score is <= 5% of the input minus baseline score.
    tf.debugging.assert_near(ig_score, (input_score - baseline_score), rtol=0.05)
    tf.print('Approximation accuracy within 5%.', output_stream=sys.stdout)
  except tf.errors.InvalidArgumentError:
    tf.print('Increase or decrease m_steps to increase approximation accuracy.', output_stream=sys.stdout)

  #tf.print('Baseline score: {:.3f}'.format(baseline_score))
  tf.print(baseline_score)
  #tf.print('Input score: {:.3f}'.format(input_score))
  #tf.print('IG score: {:.3f}'.format(ig_score))
  #tf.print('Convergence delta: {:.3f}'.format(delta))

In [ ]:
import sys

In [ ]:
convergence_check(model=model,
                  attributions=ig_attributions,
                  baseline=name_baseline_tensors['Baseline Image: Black'],
                  input=img_name_tensors['C19_603970_26'],
                  target_class_idx=0)

In [ ]:
def plot_img_attributions(model,
                          baseline,
                          img,
                          target_class_idx,
                          m_steps=50,
                          cmap=None,
                          overlay_alpha=0.4):
  """
  Args:
    model(keras.Model): A trained model to generate predictions and inspect.
    baseline(Tensor): A 3D image tensor with the shape
      (image_height, image_width, 3) with the same shape as the input tensor.
    img(Tensor): A 3D image tensor with the shape
      (image_height, image_width, 3).
    target_class_idx(Tensor): An integer that corresponds to the correct
      ImageNet class index in the model's output predictions tensor. Default
        value is 50 steps.
    m_steps(Tensor): A 0D tensor of an integer corresponding to the number of
      linear interpolation steps for computing an approximate integral.
    cmap(matplotlib.cm): Defaults to None. Reference for colormap options -
      https://matplotlib.org/3.2.1/tutorials/colors/colormaps.html. Interesting
      options to try are None and high contrast 'inferno'.
    overlay_alpha(float): A float between 0 and 1 that represents the intensity
      of the original image overlay.
  Returns:
    fig(matplotlib.pyplot.figure): fig object to utilize for displaying, saving
      plots.
  """
  # Attributions
  ig_attributions = integrated_gradients(model=model,
                          baseline=baseline,
                          input=img,
                          target_class_idx=target_class_idx,
                          m_steps=m_steps)

  convergence_check(model, ig_attributions, baseline, img, target_class_idx)

  # Per the original paper, take the absolute sum of the attributions across
  # color channels for visualization. The attribution mask shape is a greyscale image
  # with shape (224, 224).
  attribution_mask = tf.reduce_sum(tf.math.abs(ig_attributions), axis=-1)

  # Visualization
  fig, axs = plt.subplots(nrows=2, ncols=2, squeeze=False, figsize=(8, 8))

  axs[0,0].set_title('Baseline Image')
  axs[0,0].imshow(baseline)
  axs[0,0].axis('off')

  axs[0,1].set_title('Original Image')
  axs[0,1].imshow(img)
  axs[0,1].axis('off')

  axs[1,0].set_title('IG Attribution Mask')
  axs[1,0].imshow(attribution_mask, cmap=cmap)
  axs[1,0].axis('off')

  axs[1,1].set_title('Original + IG Attribution Mask Overlay')
  axs[1,1].imshow(attribution_mask, cmap=cmap)
  axs[1,1].imshow(img, alpha=overlay_alpha)
  axs[1,1].axis('off')

  plt.tight_layout()

  return fig

In [ ]:
_ = plot_img_attributions(model=model,
                          img=img_name_tensors['C19_603970_26'],
                          baseline=name_baseline_tensors['Baseline Image: Black'],
                          target_class_idx=0,
                          m_steps=20,
                          cmap=plt.cm.inferno,
                          overlay_alpha=0.4)

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow # cv2.imshow does not work on Google Colab notebooks, which is why we are using cv2_imshow instead


In [ ]:
#model = InceptionV3(weights='imagenet')
model = model_loaded
model.summary()

In [ ]:
!wget https://indiasendangered.com/wp-content/uploads/2011/09/elephant.jpg
!wget https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12234558/Chinook-On-White-03.jpg
!wget https://icatcare.org/app/uploads/2018/07/Thinking-of-getting-a-cat.png

In [ ]:
ORIGINAL = '/content/drive/My Drive/data_byclass_jan20/test/positive/MIDRC-RICORD-1C-SITE2-000265-99793-1.png'
DIM = 224

img = image.load_img(ORIGINAL, target_size=(DIM, DIM))

cv2_imshow(cv2.imread(ORIGINAL)) # Visualize image

In [ ]:
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
#print(decode_predictions(preds))

In [ ]:
print(np.argmax(preds,axis=1))

In [ ]:
with tf.GradientTape() as tape:
  last_conv_layer = model.get_layer('separable_conv2d_13')
  iterate = tf.keras.models.Model([model.inputs], [model.output, last_conv_layer.output])
  model_out, last_conv_layer = iterate(x)
  class_out = model_out[:, np.argmax(model_out[0])]
  grads = tape.gradient(class_out, last_conv_layer)
  pooled_grads = K.mean(grads, axis=(0, 1, 2))

heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)

In [ ]:
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
heatmap = heatmap.reshape((7,7))
plt.matshow(heatmap)
plt.show()

In [ ]:
img = cv2.imread(ORIGINAL)

In [ ]:
INTENSITY = 0.5

heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

heatmap = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

img = heatmap * INTENSITY + img

In [ ]:
cv2_imshow(cv2.imread(ORIGINAL))
cv2_imshow(img)

In [ ]:
#GRAD-CAM
def gradCAM(orig, intensity=0.5, res=250):
  img = image.load_img(orig, target_size=(DIM, DIM))

  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)

  preds = model.predict(x)
  print(decode_predictions(preds)[0][0][1]) # prints the class of image

  with tf.GradientTape() as tape:
    last_conv_layer = model.get_layer('separable_conv2d_43')
    iterate = tf.keras.models.Model([model.inputs], [model.output, last_conv_layer.output])
    model_out, last_conv_layer = iterate(x)
    class_out = model_out[:, np.argmax(model_out[0])]
    grads = tape.gradient(class_out, last_conv_layer)
    pooled_grads = K.mean(grads, axis=(0, 1, 2))

  heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)
  heatmap = np.maximum(heatmap, 0)
  heatmap /= np.max(heatmap)
  heatmap = heatmap.reshape((8, 8))

  img = cv2.imread(orig)

  heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

  heatmap = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

  img = heatmap * intensity + img

  cv2_imshow(cv2.resize(cv2.imread(orig), (res, res)))
  cv2_imshow(cv2.resize(img, (res, res)))

gradCAM("Chinook-On-White-03.jpg")
gradCAM("Thinking-of-getting-a-cat.png")


In [ ]:
pip install numpy==1.19.5

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow